In [1]:
import os
import sys
sys.path.append('../source/maskrcnn')
sys.path.append('../source/dataset')
sys.path.append('../source/miq')
sys.path.append('../source')
sys.path.append('C:\\Program Files\\Micro-Manager-2.0beta')

import utils
import model as modellib
import visualize
import config

import mark_dataset
import alignment
import time

import chip
import position as pos
import focus
import sc_utils

# Load a Micro-Manager instance
mmc = sc_utils.get_mmc()
print ("Stage at: (", mmc.getXPosition(), 
       ",", mmc.getYPosition(), ")")

Using TensorFlow backend.


Stage at: ( -15101.0 , 6358.5 )


In [2]:
''' Name chip and give directory to save in '''
chip_number = '0009' 
save_dir = 'C:\\Users\\cell_ml\\Desktop\\SC_WIN10\\Data\\' + chip_number
os.makedirs(save_dir, exist_ok=True)

## Auto-Alignment

In [3]:
model = alignment.get_inference_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Loading weights from  C:\Users\cell_ml\Desktop\SC_WIN10\SmartScope\python\notebook\adjusted_frame_alignment_20.h5


In [ ]:
start = time.time()

center, img, frame, r = alignment.find_alignment_mark(model)
alignment.move_to_center(mmc, center)
p1 = pos.StagePosition(x=mmc.getXPosition(),
                       y=mmc.getYPosition())
pos.set_pos(mmc, x=(p1.x - 25745.5),
                 y=p1.y)
pos.set_pos(mmc, z=focus.focus_point(mmc))

center, img, frame, r = alignment.find_alignment_mark(model)
alignment.move_to_center(mmc, center)
p2 = pos.StagePosition(x=mmc.getXPosition(),
                       y=mmc.getYPosition())
pos.set_pos(mmc, x=(p2.x + 100.0),
                 y=(p2.y - 9457.5))
pos.set_pos(mmc, z=focus.focus_point(mmc))

center, img, frame, r = alignment.find_alignment_mark(model)
alignment.move_to_center(mmc, center)
p3 = pos.StagePosition(x=mmc.getXPosition(),
                       y=mmc.getYPosition())

align_time = time.time()
print ('Time for alignment:', align_time-start)

# Create a Position List of the corners and save it
corners = pos.PositionList(positions=[p1,p2,p3])
corners.save(chip_number + '_corners', save_dir)
# Create a chip instance and visualize the focus Position List 
current_chip = chip.Chip(corners)
focus_pl = current_chip.get_focus_position_list(5,4)

focused_pl = focus.focus_from_last_point(focus_pl, mmc)
focused_pl.save(chip_number + '_focused', save_dir)

focus_time = time.time()
print ('Time for focus:', focus_time-align_time)

imaging_pl = current_chip.get_position_list(focused_pl)
imaging_pl.image(mmc, save_dir, save_jpg=False)

end = time.time()
print ('Time for imaging:', end-focus_time)
print ('Total time:', end-start)


(2200, 2688)
Processing 1 images
image                    shape: (2200, 2688, 3)       min:   16.00000  max:   64.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -39.90000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 2688.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model.ckpt-1000042
(2200, 2688)
Processing 1 images
image                    shape: (2200, 2688, 3)       min:   16.00000  max:   64.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -39.90000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 2688.00000  float6

## Load previously saved Positions

In [4]:
current_chip = chip.Chip(pos.load('\\'+chip_number + '_corners', save_dir))
focused_pl = pos.load('\\'+chip_number + '_focused', save_dir)
imaging_pl = current_chip.get_position_list(focused_pl)
imaging_pl.image(mmc, save_dir)